<a href="https://colab.research.google.com/github/Osondu-ifunanya/Wetland-classification-and-change-detection-using-Sentinel-1-2-and-RF/blob/main/WetlandClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# --------------------------
# 1) PARAMETERS / SETTINGS
# --------------------------
np.random.seed(42)
GRID_H, GRID_W = 80, 80          # spatial grid size (pixels)
N_PIXELS = GRID_H * GRID_W

# fraction of wetlands initially
WETLAND_COVER = 0.20

# fraction of wetland pixels that change (gain or loss) by T2
CHANGE_RATE = 0.10

# --------------------------
# 2) SYNTHETIC SENTINEL-1 GENERATION
# --------------------------
def generate_sar_field(h, w, vv_mean, vh_mean, vv_std=0.5, vh_std=0.3, speckle_scale=0.1):
    """Generate synthetic SAR backscatter fields (linear scale) with speckle-like noise."""
    vv = np.random.normal(vv_mean, vv_std, size=(h, w))
    vh = np.random.normal(vh_mean, vh_std, size=(h, w))
    # multiplicative speckle noise
    speckle_vv = np.random.gamma(shape=1/speckle_scale, scale=speckle_scale, size=(h, w))
    speckle_vh = np.random.gamma(shape=1/speckle_scale, scale=speckle_scale, size=(h, w))
    vv = vv * speckle_vv
    vh = vh * speckle_vh
    # clip to realistic SAR linear-range positive values
    vv = np.clip(vv, 1e-3, None)
    vh = np.clip(vh, 1e-3, None)
    return vv, vh

# Create base landcover mask: 1 = wetland, 0 = non-wetland
base_mask = np.zeros((GRID_H, GRID_W), dtype=int)
n_wet = int(WETLAND_COVER * N_PIXELS)
wet_indices = np.random.choice(N_PIXELS, size=n_wet, replace=False)
base_mask.flat[wet_indices] = 1

# Generate SAR at T1
# Assume wetlands have lower VV (water absorption) and different VH behavior
vv_t1, vh_t1 = generate_sar_field(GRID_H, GRID_W,
                                  vv_mean=np.where(base_mask==1, 2.5, 4.5),
                                  vh_mean=np.where(base_mask==1, 0.9, 1.6),
                                  vv_std=0.3, vh_std=0.15, speckle_scale=0.08)

# Create T2 landcover by applying random changes
t2_mask = base_mask.copy()
n_change = int(CHANGE_RATE * N_PIXELS)
change_indices = np.random.choice(N_PIXELS, size=n_change, replace=False)
# flip some pixels: wet->non-wet or non-wet->wet with 50/50
for idx in change_indices:
    if np.random.rand() < 0.5:
        t2_mask.flat[idx] = 1 - t2_mask.flat[idx]  # toggle

# Generate SAR at T2 using t2_mask-dependent stats
vv_t2, vh_t2 = generate_sar_field(GRID_H, GRID_W,
                                  vv_mean=np.where(t2_mask==1, 2.7, 4.3),
                                  vh_mean=np.where(t2_mask==1, 0.95, 1.55),
                                  vv_std=0.35, vh_std=0.16, speckle_scale=0.09)

# --------------------------
# 3) FEATURE ENGINEERING
# --------------------------
# Use log10(dB-like) transforms commonly used with SAR (just for synthetic realism)
def to_db(x):
    # ensure positive
    return 10.0 * np.log10(np.clip(x, 1e-6, None))

vv_t1_db = to_db(vv_t1); vh_t1_db = to_db(vh_t1)
vv_t2_db = to_db(vv_t2); vh_t2_db = to_db(vh_t2)

# Ratio and difference features (VV/VH, VV-VH, temporal difference)
eps = 1e-6
vv_vh_ratio_t1 = vv_t1 / (vh_t1 + eps)
vv_vh_ratio_t2 = vv_t2 / (vh_t2 + eps)
vv_minus_vh_t1 = vv_t1_db - vh_t1_db
vv_minus_vh_t2 = vv_t2_db - vh_t2_db

# Temporal changes
delta_vv_db = vv_t2_db - vv_t1_db
delta_vh_db = vh_t2_db - vh_t1_db
delta_ratio = vv_vh_ratio_t2 - vv_vh_ratio_t1
delta_vv_minus_vh = vv_minus_vh_t2 - vv_minus_vh_t1

# Stack feature arrays into per-pixel rows
features_t1 = np.stack([
    vv_t1_db.flatten(),
    vh_t1_db.flatten(),
    np.log1p(vv_vh_ratio_t1.flatten()),
    vv_minus_vh_t1.flatten()
], axis=1)

features_t2 = np.stack([
    vv_t2_db.flatten(),
    vh_t2_db.flatten(),
    np.log1p(vv_vh_ratio_t2.flatten()),
    vv_minus_vh_t2.flatten()
], axis=1)

features_delta = np.stack([
    delta_vv_db.flatten(),
    delta_vh_db.flatten(),
    np.log1p(np.abs(delta_ratio.flatten())+eps),
    delta_vv_minus_vh.flatten()
], axis=1)

# Ground truth labels per pixel
y_t1 = base_mask.flatten()
y_t2 = t2_mask.flatten()

# --------------------------
# 4) CLASSIFICATION (Random Forest)
# --------------------------
# Train RF on T1 features to classify wetlands at T1
X_train, X_test, y_train, y_test = train_test_split(features_t1, y_t1, test_size=0.25, random_state=0, stratify=y_t1)

rf = RandomForestClassifier(n_estimators=200, random_state=0, min_samples_leaf=5)
rf.fit(X_train, y_train)

y_test_pred = rf.predict(X_test)
print("T1 Classification Report (on test subset):")
print(classification_report(y_test, y_test_pred, digits=3))
print("Accuracy:", accuracy_score(y_test, y_test_pred))

# Predict full maps at T1 and T2
pred_t1 = rf.predict(features_t1).reshape(GRID_H, GRID_W)
pred_t2 = rf.predict(features_t2).reshape(GRID_H, GRID_W)

# --------------------------
# 5) CHANGE DETECTION
# --------------------------
# Simple rule: changed where predicted class differs between T1 and T2
change_map_pred = (pred_t1 != pred_t2).astype(int)
change_map_true = (base_mask != t2_mask).astype(int)

# --------------------------
# 6) EVALUATION
# --------------------------
print("\nConfusion matrix for T1 (test subset):")
print(confusion_matrix(y_test, y_test_pred))

# Evaluate change detection (pixel-wise)
print("\nChange detection evaluation (pixel-wise):")
print("True change fraction:", change_map_true.mean())
print("Predicted change fraction:", change_map_pred.mean())
print("Change detection classification report (predicted vs true):")
print(classification_report(change_map_true.flatten(), change_map_pred.flatten(), digits=3))

# --------------------------
# 7) VISUALIZATIONS
# --------------------------
plt.figure(figsize=(12,8))

plt.subplot(3,3,1); plt.title("VV T1 (dB)"); plt.imshow(vv_t1_db, cmap='viridis'); plt.colorbar()
plt.subplot(3,3,2); plt.title("VH T1 (dB)"); plt.imshow(vh_t1_db, cmap='viridis'); plt.colorbar()
plt.subplot(3,3,3); plt.title("VV/VH ratio T1 (log)"); plt.imshow(np.log1p(vv_vh_ratio_t1), cmap='magma'); plt.colorbar()

plt.subplot(3,3,4); plt.title("True Wetland T1"); plt.imshow(base_mask, cmap='Blues'); plt.colorbar()
plt.subplot(3,3,5); plt.title("Predicted Wetland T1"); plt.imshow(pred_t1, cmap='Blues'); plt.colorbar()
plt.subplot(3,3,6); plt.title("Predicted Wetland T2"); plt.imshow(pred_t2, cmap='Blues'); plt.colorbar()

plt.subplot(3,3,7); plt.title("True Change (T1→T2)"); plt.imshow(change_map_true, cmap='Reds'); plt.colorbar()
plt.subplot(3,3,8); plt.title("Predicted Change (T1→T2)"); plt.imshow(change_map_pred, cmap='Reds'); plt.colorbar()
plt.subplot(3,3,9); plt.title("Delta VV dB (T2-T1)"); plt.imshow(delta_vv_db, cmap='coolwarm'); plt.colorbar()

plt.tight_layout()
plt.show()

# --------------------------
# 8) EXPORT RESULTS TO EXCEL (per-pixel)
# --------------------------
df_export = pd.DataFrame({
    'pixel_id': np.arange(N_PIXELS),
    'row': np.repeat(np.arange(GRID_H), GRID_W),
    'col': np.tile(np.arange(GRID_W), GRID_H),
    'VV_T1_dB': vv_t1_db.flatten(),
    'VH_T1_dB': vh_t1_db.flatten(),
    'VV_T2_dB': vv_t2_db.flatten(),
    'VH_T2_dB': vh_t2_db.flatten(),
    'VVVH_ratio_T1_log': np.log1p(vv_vh_ratio_t1.flatten()),
    'VVVH_ratio_T2_log': np.log1p(vv_vh_ratio_t2.flatten()),
    'delta_VV_dB': delta_vv_db.flatten(),
    'delta_VH_dB': delta_vh_db.flatten(),
    'true_wetland_T1': y_t1,
    'true_wetland_T2': y_t2,
    'pred_wetland_T1': rf.predict(features_t1),
    'pred_wetland_T2': rf.predict(features_t2),
    'true_change': change_map_true.flatten(),
    'pred_change': change_map_pred.flatten()
})

excel_path = "synthetic_s1_wetland_change_detection.xlsx"
df_export.to_excel(excel_path, index=False)
print(f"\nExported per-pixel results to: {excel_path}")

# --------------------------
# 9) FEATURE IMPORTANCE
# --------------------------
feat_names = ['VV_dB', 'VH_dB', 'log(VV/VH)', 'VV_minus_VH_dB']
fi = rf.feature_importances_
fi_df = pd.DataFrame({'feature': feat_names, 'importance': fi}).sort_values('importance', ascending=False)
print("\nFeature importances (RF for T1):")
print(fi_df)
